#### Added pipeline_name and tokenizer --(working)

In [1]:
from a360mdk import A360AI

In [2]:
datarepos = a360ai.list_datarepos()
datarepos

,name,description,storage_provider
0,Jupyter R-Python,Test Jupyter R-Python notebook image,aws


In [1]:
a360ai.set_default_datarepo("Jupyter R-Python")

In [4]:
a360ai.list_datasets()

,base_name,extension,size
1,SMSSpamCollection,,483481.0
2,SMSSpamCollection.csv,csv,500350.0
3,df.parquet,parquet,325696.0


In [2]:
df = a360ai.load_dataset("df.parquet")
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
import pandas as pd
X=list(df['message'])[:100]
y=list(df['label'])[:100]
y=list(pd.get_dummies(y,drop_first=True)['spam'])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [4]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True,max_length=512)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=512)

In [5]:
import torch
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)

In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
10,0.632200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15, training_loss=0.6285655498504639, metrics={'train_runtime': 12.8129, 'train_samples_per_second': 18.731, 'train_steps_per_second': 1.171, 'total_flos': 4036116052800.0, 'train_loss': 0.6285655498504639, 'epoch': 3.0})

In [7]:
import pandas as pd
X_train = pd.DataFrame(X_train)
X_train.columns =['message']
y_train = pd.DataFrame(y_train)
y_train.columns =['label']
y_train

,label
0,0
1,0
2,0
3,0
4,0
...,...
75,0
76,1
77,0
78,0


In [8]:
my_model = a360ai.get_or_create_model(model_name=f"huggingfacetest2", version="1.0.1")

/home/jovyan/a360-mdk/a360mdk/controllers/model.py:220: UserWarning: Only the first 15 characters of the model name will be used for deployment and deployment monitoring
  warnings.warn(


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/vocab.txt
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--distilber

In [10]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [11]:
experiment = my_model.get_or_create_experiment(
    experiment_name = f"huggingfacetest2",
    model_flavor = "huggingface",
    enable_drift_monitoring=False,
    train_features=X_train,
    train_target=y_train,
    feature_names=list(["message"]),
    data_exploration_file="/home/jovyan/01_exploratory-data-analysis.ipynb",
    data_preparation_file="/home/jovyan/02_data-preprocessing.ipynb",
    model_training_file="/home/jovyan/03_model-training.ipynb",
    pipeline_name = "text-classification",
    tokenizer = tokenizer,
    framework = "pt"
    
)


                //-- Experiment Loaded --//
                Model Name: huggingfacetest2
                Model Version: 1.0.1
                Experiment Name: huggingfacetest2
                Final Run Id: None
                Model Flavor: huggingface
                Input Signature: message text
                Output Signature: label big_integer
                Data Exploration File: /home/jovyan/01_exploratory-data-analysis.ipynb
                Data Preparation File: /home/jovyan/02_data-preprocessing.ipynb
                Model Training File: /home/jovyan/03_model-training.ipynb
                Drift Monitoring Enabled: False
                Pipe Line: text-classification


In [12]:
test = trainer.evaluate(test_dataset)['eval_loss']
train = trainer.evaluate(train_dataset)['eval_loss']

***** Running Evaluation *****
  Num examples = 20
  Batch size = 64


***** Running Evaluation *****
  Num examples = 80
  Batch size = 64


In [13]:
test

0.6450785398483276

In [14]:
train

0.6113789081573486

In [15]:
metrics = {'test_score': test, 'train_score': train}

In [16]:
metrics

{'test_score': 0.6450785398483276, 'train_score': 0.6113789081573486}

In [17]:
with experiment.run_experiment() as run:
    run.log_metrics(metrics)
    run.log_model(model)

/home/jovyan/a360-mdk/a360mdk/controllers/notebook_save.py:93: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cells = cells.append(final_input_cell, ignore_index=True)


Run - 8ab7b8d8-6cd3-49a3-aa7c-305fc1fc183f/trained_model
ONNX opset version set to: 12
Loading pipeline (model: DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e

/opt/conda/lib/python3.10/site-packages/transformers/convert_graph_to_onnx.py:380: FutureWarning: The `transformers.convert_graph_to_onnx` package is deprecated and will be removed in version 5 of Transformers
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:213: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [18]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jovyan


In [19]:
runs = experiment.list_runs()
runs

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
0,8ab7b8d8-6cd3-49a3-aa7c-305fc1fc183f,85f93462-f8d5-49bd-ba1d-bd415b5c754c,[{'id': '75caba35-d04b-4a3c-bbe1-222da8d42398'...,0.645079,0.611379,0.000009


In [20]:
runs.sort_values('metric_test_score',ascending=False)

,id,dataset,artifact_paths,metric_test_score,metric_train_score,metadata_run_time
0,8ab7b8d8-6cd3-49a3-aa7c-305fc1fc183f,85f93462-f8d5-49bd-ba1d-bd415b5c754c,[{'id': '75caba35-d04b-4a3c-bbe1-222da8d42398'...,0.645079,0.611379,0.000009


In [21]:
best_run_id = runs.sort_values("metric_test_score", ascending=False).id.values[0]
best_run_id

'8ab7b8d8-6cd3-49a3-aa7c-305fc1fc183f'

In [22]:
experiments = my_model.list_experiments()
experiments

,id,bestRun,model_id,experiment_name,best_run_id,model_flavor,input_signature,output_signature,data_exploration_file,data_preparation_file,model_training_file,baseline,train_shape,model_name,version,updated_at
0,7b7e1daa-e8f5-4895-b472-461890a18466,None,559cf658-2398-48b5-9f29-23ea3e5ab39c,huggingfacetest2,None,huggingface,message text,label big_integer,/home/jovyan/01_exploratory-data-analysis.ipynb,/home/jovyan/02_data-preprocessing.ipynb,/home/jovyan/03_model-training.ipynb,{},[1],huggingfacetest2,1.0.1,2022-10-19 14:13


In [23]:
my_model.set_final_run(experiment, best_run_id)

In [24]:
my_model.get_artifact("trained_model",path="/home/jovyan")

In [25]:
from a360runtime import ONNXModel

In [26]:
path = "/home/jovyan/trained_model"
model = ONNXModel(path)

In [27]:
input_data = "this is some good string"

In [28]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
input = tokenizer(input_data, return_tensors="pt")
# # ONNX Runtime expects NumPy arrays
inputs = {k: v.numpy() for k, v in input.items()}


loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/jovyan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/vocab.txt
loading file tokenizer.json from cache at /home/jovyan/.cache/huggingface/hub/models--distilber

In [29]:
from scipy.special import softmax
logits_onnx = model.predict(inputs , "huggingface")
logits_onnx
preds = softmax(logits_onnx[0][0])
print(f"Class: {['ham' if preds.argmax() == 0 else 'spam'][0]}, Probability: {preds.max():.4f}")


Class: ham, Probability: 0.5664
